In [1]:
import pymeshlab
import numpy as np

In [2]:
ms = pymeshlab.MeshSet()

In [3]:
ms.load_new_mesh("model_normalized.obj")

In [4]:
m = ms.current_mesh()

In [5]:
v_matrix = m.vertex_matrix()

In [6]:
f_matrix = m.face_matrix()

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [7]:
# Define the maximum order of moments to be calculated
max_m = 5

In [8]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

In [9]:
def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)
    

In [10]:
def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    

In [11]:
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


In [12]:
def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reqused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



In [13]:
moments = M_ijk(f_matrix,v_matrix,max_m)

In [14]:
print(moments)

[[[-1.98788045e-20  2.92615142e-19 -3.18467638e-20  3.59965738e-21
   -8.04685258e-21]
  [ 7.66589633e-21  9.39987369e-21  2.29909681e-21  9.28716225e-23
   -1.45913277e-21]
  [-2.60303399e-23 -6.53020320e-22 -4.44536878e-23 -1.02610457e-22
    0.00000000e+00]
  [ 1.41627215e-22  6.63971981e-23 -1.84201167e-23  0.00000000e+00
    0.00000000e+00]
  [ 4.40449442e-24 -1.60183319e-23  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 2.54016826e-21 -5.20477551e-23 -4.58390537e-23  5.68500879e-23
    1.66961464e-23]
  [-7.89763615e-23  2.63057975e-24 -7.89199169e-24 -3.66437881e-24
    0.00000000e+00]
  [-2.21618165e-24  1.00555153e-24  4.95650773e-25  0.00000000e+00
    0.00000000e+00]
  [ 1.24319430e-24 -1.16382227e-25  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 7.53803927e-26  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[-4.26256509e-22  2.18435458e-23 -3.87191865e-24 -1.11936996e-25
    0.00000000e+00]
  [-8.90072390e-24 -5.02409733e-26  1.0